# Test for data perbutation

In [107]:
import pandas as pd
import numpy as np
import os, glob, torch, random, tqdm
from torch_geometric.data import Data, InMemoryDataset, DataLoader
from torch_geometric.utils import degree, add_self_loops, convert
# import torch
from BiGCN.Process.dataset import GraphDataset,BiGraphDataset,UdGraphDataset
os.chdir('/home/rita/111/111-2MLG/project/BiGCN')
from BiGCN.Process.process import *
from BiGCN.Process.rand5fold import *
# os.chdir('/home/rita/111/111-2MLG/project')
os.getcwd()

'/home/rita/111/111-2MLG/project/BiGCN'

In [102]:
def set_init(graph) :
    row, col = graph.edge_index
    row = row.unsqueeze(0)
    col = col.unsqueeze(0)
    t = torch.hstack([row, col])
    t = t.squeeze()

    # deg = degree(t).float().unsqueeze(1)
    deg = degree(t).int().unsqueeze(1)
    extend = torch.ones(graph.num_nodes, 2)
    init = torch.hstack([deg, extend])
    graph.x = init
    
    return graph

In [183]:
lr=0.0005
weight_decay=1e-4
patience=10
# n_epochs=200
n_epochs=2
batchsize=128
TDdroprate=0.2
BUdroprate=0.2
# datasetname=sys.argv[1] #"Twitter15"、"Twitter16"
datasetname="Twitter15"
# iterations=int(sys.argv[2])
treeDic=loadTree(datasetname)

fold0_x_test, fold0_x_train, \
    fold1_x_test,  fold1_x_train,  \
    fold2_x_test, fold2_x_train, \
    fold3_x_test, fold3_x_train, \
    fold4_x_test,fold4_x_train = load5foldData(datasetname)
    
    
traindata_list, testdata_list = loadBiData(datasetname, treeDic, fold0_x_test, fold0_x_train, TDdroprate,BUdroprate)

print('\n', len(traindata_list))
print(len(testdata_list))
print(type(traindata_list))
print(type(traindata_list[0]))
print(traindata_list[0])
print(traindata_list[0].edge_index[0].unique())

reading twitter tree
tree no: 3098
/home/rita/111/111-2MLG/project/BiGCN/
loading tree label
1490
374 370 372 374
loading train set
train no: 295
loading test set
test no: 1175

 295
1175
<class 'dataset.BiGraphDataset'>
<class 'torch_geometric.data.data.Data'>
Data(x=[64, 5000], edge_index=[2, 50], y=[1], BU_edge_index=[2, 50], root=[1, 5000], rootindex=[1])
tensor([0, 1, 3, 4, 5])


In [166]:
# %load ./Process/perturbation.py
# import pandas as pd
import numpy as np
import os, glob, torch, random
# from torch_geometric.data import Data, InMemoryDataset, DataLoader
# from torch_geometric.utils import degree, add_self_loops, convert
# import torch
# from BiGCN.Process.dataset import GraphDataset,BiGraphDataset,UdGraphDataset
# os.chdir('/home/rita/111/111-2MLG/project/BiGCN')
# from BiGCN.Process.process import *
# from BiGCN.Process.rand5fold import *

def pertubation(data, method, rho = 0.1, BU = False) :
    data.to('cpu')
    n_nodes, n_feats = data.x.shape
    choice = np.arange(n_nodes)
    choice = choice[choice != np.array(data.rootindex)]
    selected = np.random.choice(choice, np.around(n_nodes * rho).astype(int), replace=False)
    if method == 'CN' : # Comments contain noise
        noise = np.zeros((n_nodes, n_feats))
        noise[selected] = np.random.randn(1, n_feats)
        noise = torch.from_numpy(noise)
        data.x += noise
        
    elif method == 'CD' : # Comments are deleted
        data.x[selected] = 0
        
    elif method == 'CE' and selected.shape[0] != 1: # Comments are exchangeable
        exchange = selected[torch.randperm(selected.shape[0]).clone()]
        data.x[selected] = data.x[exchange]
        
    elif method == 'PR' : # Propagation sub-structure is removed
        choice = data.edge_index[0].unique()
        choice = choice[choice != data.rootindex]
        if choice.nelement() != 0 :
            selected = np.random.choice(choice, 1)
            idx = (data.edge_index[0] == selected[0]).unsqueeze(0)
            idx = idx.repeat(2, 1)
            data.edge_index = torch.masked_select(data.edge_index, ~idx).reshape(2, -1)
            
            # BU
            if BU :
                idx = (data.BU_edge_index[1] == selected[0]).unsqueeze(0)
                idx = idx.repeat(2, 1)
                data.BU_edge_index = torch.masked_select(data.BU_edge_index, ~idx).reshape(2, -1)
        
    elif method == 'PU' : # Propagation structure is uncertain 
        n = data.edge_index.shape[1]
        choice = np.random.choice(n, np.around(n * rho).astype(int), replace = False)
        idx = np.arange(n) == -1
        idx[choice] = True
        # del_idx = [[i, j] for i, j in zip(data.edge_index[0][choice], data.edge_index[1][choice])]
        # print(del_idx)
        idx = torch.from_numpy(idx).unsqueeze(0)
        idx = idx.repeat(2, 1)
        data.edge_index = torch.masked_select(data.edge_index, ~idx).reshape(2, -1)
        
        # BU
        if BU:
            n = data.BU_edge_index.shape[1]
            choice = np.random.choice(n, np.around(n * rho).astype(int), replace = False)
            idx = np.arange(n) == -1
            idx[choice] = True
            
            # del_idx = [[i, j] for i, j in zip(data.edge_index[0][choice], data.edge_index[1][choice])]
            # print(del_idx)
            idx = torch.from_numpy(idx).unsqueeze(0)
            idx = idx.repeat(2, 1)
            data.BU_edge_index = torch.masked_select(data.BU_edge_index, ~idx).reshape(2, -1)
        
    elif method == 'PI' : # Propagation structure is incorrect
        choice = torch.from_numpy(choice)
        if choice.nelement() > 1 :
            selected = np.random.choice(choice, 2, replace=False)
            # print(selected)
            # del i's parent
            idx = (data.edge_index[1] == selected[0]).unsqueeze(0)
            idx = idx.repeat(2, 1)
            data.edge_index = torch.masked_select(data.edge_index, ~idx).reshape(2, -1)
            # connect i and j
            temp1 = data.edge_index[0].reshape(1, -1)
            temp1 = torch.cat((temp1, torch.tensor(selected[0]).reshape(1, -1)), 1)
            
            temp2 = data.edge_index[1].reshape(1, -1)
            temp2 = torch.cat((temp2, torch.tensor(selected[1]).reshape(1, -1)), 1)
            
            data.edge_index = torch.cat((temp1, temp2), 0)
            
            # BU
            if BU :
                selected = np.random.choice(choice, 2, replace=False)
                # print(selected)
                # del i's parent
                idx = (data.BU_edge_index[1] == selected[0]).unsqueeze(0)
                idx = idx.repeat(2, 1)
                data.BU_edge_index = torch.masked_select(data.BU_edge_index, ~idx).reshape(2, -1)
                # connect i and j
                temp1 = data.BU_edge_index[0].reshape(1, -1)
                temp1 = torch.cat((temp1, torch.tensor(selected[0]).reshape(1, -1)), 1)
                
                temp2 = data.BU_edge_index[1].reshape(1, -1)
                temp2 = torch.cat((temp2, torch.tensor(selected[1]).reshape(1, -1)), 1)
                
                data.BU_edge_index = torch.cat((temp1, temp2), 0)
    # data.to('cuda:0')
    return data
print(traindata_list[0])
temp = pertubation(traindata_list[0], 'PU')
print(temp)


Data(x=[79, 5000], edge_index=[2, 62], y=[1], BU_edge_index=[2, 62], root=[1, 5000], rootindex=[1])
Data(x=[79, 5000], edge_index=[2, 56], y=[1], BU_edge_index=[2, 62], root=[1, 5000], rootindex=[1])


In [228]:
temp = traindata_list[0]
a = np.array(temp.y)
b = np.array([0, 1, 2, 3])
print(type(a))
print(type(b))
print(a == b)
print(b[a != b])
print(np.random.choice(b[a != b], 1))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[False False False  True]
[0 1 2]
[1]


In [252]:
def change_label(data, p) :
    p = np.random.choice([0, 1], p = [1-p, p])
    print('p = ', p)
    temp = data
    if p :
        a = np.array(temp.y)
        b = np.array([0, 1, 2, 3])
        c = np.random.choice(b[a != b], 1)
        print(data.y)
        print(c)
        temp.y = torch.LongTensor(c)
    
    return temp


temp = traindata_list[0]
print(temp, temp.y, sep = '\n')
temp = change_label(temp, 0.1)
print(temp, temp.y, sep = '\n')

Data(x=[64, 5000], edge_index=[2, 50], y=[1], BU_edge_index=[2, 50], root=[1, 5000], rootindex=[1])
tensor([3])
p =  0
Data(x=[64, 5000], edge_index=[2, 50], y=[1], BU_edge_index=[2, 50], root=[1, 5000], rootindex=[1])
tensor([3])


In [176]:
train_loader = DataLoader(traindata_list, batch_size=batchsize, shuffle=True, num_workers=5)
test_loader = DataLoader(testdata_list, batch_size=batchsize, shuffle=True, num_workers=5)
avg_loss = []
avg_acc = []
batch_idx = 0
tqdm_train_loader = tqdm.tqdm(train_loader)
for Batch_data in tqdm_train_loader:
    # print(Batch_data.batch)
    # print(Batch_data.y)
    for i in range(len(Batch_data)) :
        print(Batch_data[i])
        print(Batch_data[i].y)
    # try :
    #     print(Batch_data.batch)
    # except AttributeError :
    #     print('erroe')

100%|██████████| 3/3 [00:00<00:00,  3.24it/s]

Data(x=[3, 5000], edge_index=[2, 1], y=[1], BU_edge_index=[2, 1], root=[1, 5000], rootindex=[1])
tensor([2])
Data(x=[22, 5000], edge_index=[2, 16], y=[1], BU_edge_index=[2, 16], root=[1, 5000], rootindex=[1])
tensor([1])
Data(x=[12, 5000], edge_index=[2, 8], y=[1], BU_edge_index=[2, 8], root=[1, 5000], rootindex=[1])
tensor([3])
Data(x=[61, 5000], edge_index=[2, 48], y=[1], BU_edge_index=[2, 48], root=[1, 5000], rootindex=[1])
tensor([2])
Data(x=[77, 5000], edge_index=[2, 60], y=[1], BU_edge_index=[2, 60], root=[1, 5000], rootindex=[1])
tensor([1])
Data(x=[36, 5000], edge_index=[2, 28], y=[1], BU_edge_index=[2, 28], root=[1, 5000], rootindex=[1])
tensor([1])
Data(x=[72, 5000], edge_index=[2, 56], y=[1], BU_edge_index=[2, 56], root=[1, 5000], rootindex=[1])
tensor([0])
Data(x=[37, 5000], edge_index=[2, 28], y=[1], BU_edge_index=[2, 28], root=[1, 5000], rootindex=[1])
tensor([3])
Data(x=[11, 5000], edge_index=[2, 8], y=[1], BU_edge_index=[2, 8], root=[1, 5000], rootindex=[1])
tensor([3])

In [253]:
l = []
for i in traindata_list :
    l.append(i.y)
    # print(i.y)
print(l)
print(torch.unique(torch.concat(l)))

[tensor([3]), tensor([2]), tensor([3]), tensor([2]), tensor([2]), tensor([2]), tensor([3]), tensor([2]), tensor([2]), tensor([1]), tensor([3]), tensor([2]), tensor([0]), tensor([3]), tensor([3]), tensor([0]), tensor([0]), tensor([0]), tensor([2]), tensor([0]), tensor([1]), tensor([2]), tensor([3]), tensor([0]), tensor([3]), tensor([3]), tensor([0]), tensor([3]), tensor([1]), tensor([3]), tensor([2]), tensor([0]), tensor([2]), tensor([3]), tensor([1]), tensor([0]), tensor([3]), tensor([1]), tensor([1]), tensor([3]), tensor([0]), tensor([2]), tensor([0]), tensor([3]), tensor([3]), tensor([3]), tensor([1]), tensor([3]), tensor([0]), tensor([2]), tensor([2]), tensor([0]), tensor([0]), tensor([0]), tensor([3]), tensor([2]), tensor([2]), tensor([1]), tensor([2]), tensor([2]), tensor([2]), tensor([2]), tensor([0]), tensor([3]), tensor([0]), tensor([1]), tensor([0]), tensor([3]), tensor([1]), tensor([3]), tensor([3]), tensor([0]), tensor([1]), tensor([1]), tensor([0]), tensor([3]), tensor([0])